In [1]:
using LinearAlgebra
using Random
rng = MersenneTwister(5);

Random.seed!(rng, 69)

n = 5;

G = zeros(Float64,n,n)
    for i=1:n    
    G[i,i] = rand(rng, 1:5)                 # positive diagonals
    G[i+1:n, i] = rand(rng, -10:10, n-i)    # G is lower triangular
end

A = G * G'
display(A)

for eigval in eigvals(A)
    @assert eigval > 0
end

x_orig = rand(rng, 0:10, n)
b = A * x_orig
display(x_orig')
display(b')

5×5 Matrix{Float64}:
  25.0    0.0   45.0   10.0  -50.0
   0.0    4.0  -14.0    8.0   -8.0
  45.0  -14.0  134.0   -4.0  -82.0
  10.0    8.0   -4.0   38.0  -93.0
 -50.0   -8.0  -82.0  -93.0  301.0

1×5 adjoint(::Vector{Int64}) with eltype Int64:
 3  2  6  8  5

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 175.0  -52.0  469.0  -139.0  103.0

In [2]:
# Cholesky decomposition, num flops = 1/3 n^3 (classic LU is 2/3 n^3, LU with pivoting is n^3)

function solve_cholesky(A)
    n = size(A, 1)
    for j=1:n
        for k=1:j-1
            for i=j:n
                A[i,j] = A[i,j] - A[i,k] * A[j,k]
            end
        end

        sqrt_ajj = sqrt(A[j,j])

        for i=j:n
            A[i,j] /= sqrt_ajj
        end
    end    
    return A
end
;

In [3]:
A_fact = solve_cholesky(A)

L = tril(A_fact)
@assert norm(L - G) == 0

In [4]:
# function to solve for x, using cholesky factorized A

function solve(A_fact, b)
    x = copy(b)
    
    # Solve using matrix G
    for j=1:n
        x[j] /= A_fact[j,j]
        for i=j+1:n
            x[i] -= A_fact[i,j] * x[j]
        end
    end
    
    # Solve using matrix G^T
    for i=n:-1:1
        for j=i+1:n
            x[i] -= A_fact[j,i] * x[j]
        end
        x[i] /= A_fact[i,i]
    end
    return x
end

x_sol = solve(A_fact, b)
@assert norm(x_sol - x_orig) == 0

# Speed Comparison

In [5]:
# previous LU factorization code

function solve_col(A)
    n = size(A,1)

    for k=1:n
        @assert A[k,k] != 0
        for j=k+1:n
            for i=k+1:n
                A[i,k] /= A[k,k]
                A[i,j] -= A[i,k] * A[k,j]
            end
        end
    end
    return A
end
;

In [6]:
using LinearAlgebra
using Random
rng = MersenneTwister(5);

Random.seed!(rng, 69)

n = 500;

G = zeros(Float64,n,n)
for i=1:n    
    G[i,i] = rand(rng, 1:5)                 # positive diagonals
    G[i+1:n, i] = rand(rng, -10:10, n-i)    # G is lower triangular
end

A = G * G'
A_orig = copy(A)
;

In [8]:
A = copy(A_orig)
@time solve_cholesky(A)
A = copy(A_orig)
@time solve_col(A)
;

  0.029877 seconds
  0.160735 seconds
